# CNN: Export frames:

The following jupyter notebook would contain a class, that would use the frameseriers loader dataloader, in order to export the following:
1. Create a folder of frames.
2. Inside frames, create a folder for every chute.
3. For every chute, create 8 different folder for each camera.
3. For every camera, create two folders (fall, no_fall)
4. Inside each folder, save 5 frames (from a specific index set by user) to save 5 optical flow gradients.
5. Inside each folder, save 5 frames (from a specific index set by user) to save 5 orig images.

In [6]:
import pandas as pd
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import utils
from tqdm import tqdm

In [7]:
# See of Data path exists
RAW_DATA_PATH = os.path.join(os.getcwd(), os.pardir, "data", "MultiCamFall")
print(os.path.exists(RAW_DATA_PATH))

VIDEO_PATH = os.path.join(RAW_DATA_PATH, "dataset", "dataset")
print(os.path.exists(VIDEO_PATH))\

OF_DATA_PATH = os.path.join(os.getcwd(), "images")
print(os.path.exists(OF_DATA_PATH))

True
True
True


In [8]:
#lets obtain the metadata file
df_metadata = pd.read_csv(os.path.join(RAW_DATA_PATH, "data_tuple3.csv"))
df_metadata.loc[df_metadata["cam"] == 55, "cam"] = 3.0

In [9]:
# Get the device if cuda or not
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### 1. Use the Series dataloader in order to see if its ready for usage.

In [10]:
#Create datasets and dataloaders
test_dataset = utils.FigDatasetSeries(data_path=OF_DATA_PATH,\
    dataframe=df_metadata, device=device)


### 2. Lets create the class that would perform the setup we need, and then with the dataloader; obtain the required frames.
Now, lets create the class that would do the setup

In [54]:
from PIL import Image

class ExportFrames:
    '''
    Class that would export the necessary frames, as well
    as setup the folder to export these frames.
    '''
    def __init__(self, dataframe: pd.DataFrame, \
            dataloader:utils.FigDatasetSeries, export_csv_path:str,
            data_path:str, frame_idx_start:int = 15, frames_export:int = 5):
        '''
        Parameters:
            dataframe(pd.Dataframe): Pandas dataframe that would be used for metadata of files.
            dataloader(utils.FigDatasetSeries): Dataloader that would laod all required images.
            export_csv(str): Export csv path where we want to export specific frames.
            data_path(str): Data path where all of the OFs and orig images are stored.
            frame_idx_start(int): number between 0 to 30, where would index which frames to get.
            frames_export(int): Number of frames to export, per chute, per cam.
        '''
        self.dataframe = dataframe
        self.dataloader = dataloader
        self.export_csv_path = export_csv_path
        self.data_path = data_path
        self.frame_idx_start = frame_idx_start
        self.frames_export = frames_export

        #Obtain all self.chutes and cameras
        self.chutes = df_metadata["chute"].unique().astype(np.uint8)
        self.cam_range = df_metadata["cam"].unique().astype(np.uint8)
        ...

    def export_frames(self):
        '''
        Instance method that activates everything to export the specific frames
        '''
        self.setup_folders()
        self.iterate_data()

    def setup_folders(self):
        '''
        Instance method that would setup the folder when necessary
        '''

        #Set up the foldert to export everything
        exp_path = os.path.join(self.export_csv_path, "export")
        if os.path.exists(exp_path) == False:
            os.mkdir(exp_path)

        for chute in self.chutes:
            #Have a chute path, and check if exists to create
            chute_path = os.path.join(exp_path, f"chute_{str(chute)}")
            if os.path.exists(chute_path) == False:
                os.mkdir(chute_path)
            for cam in self.cam_range:
                #Have a cam path, and check if exists to create
                cam_path = os.path.join(chute_path, f"cam_{str(cam)}")
                if os.path.exists(cam_path) == False:
                    os.mkdir(cam_path)
                # Have a fall and no_fall paths and check if exists to create
                fall_path = os.path.join(cam_path, "fall")
                no_fall_path = os.path.join(cam_path, "no_fall")
                if os.path.exists(fall_path) == False:
                    os.mkdir(fall_path)
                if os.path.exists(no_fall_path) == False:
                    os.mkdir(no_fall_path)
    
    def iterate_data(self) -> None:
        '''
        Method that exports the data to their respective folders
        '''

        for chute in self.chutes:
            # Select the df_subset_chute
            df_subset_chute = self.dataframe[self.dataframe["chute"] == chute]
            for cam in self.cam_range:
                
                #Now lets select by cam
                df_subset_cam = df_subset_chute[df_subset_chute["cam"] == cam]

                # Select from df_subset_chute, df_fall, df_no_fall
                df_fall:pd.DataFrame = df_subset_cam[df_subset_cam["label"] == 1]
                df_no_fall:pd.DataFrame = df_subset_cam[df_subset_cam["label"] == 0]

                #Select only the first row of df_fall (we just want one scenario)
                row_fall = df_fall.iloc[0]
                row_no_fall = df_no_fall.iloc[0]

                #Now obtain the frames
                self.read_row_frames(row_fall)
                self.read_row_frames(row_no_fall)
    
    def read_row_frames(self, row:pd.DataFrame):
        '''
        Instance method that would obtain the row frames,
        and download them respectively.
        Parameters:
            row(pd.DataFrame): Dataframe row that would retrieve the
            frames that would be used for for download.
        '''
        #Obtain the important information
        chute = int(row['chute'])
        cam = int(row['cam'])
        start = int(row['start'])
        end = int(row['end'])
        label = int(row["label"])

        #Lets now see which label, to access the corresponding dir inside data path
        if label == 1:
            retrieval_path = os.path.join(self.data_path, "fall")
            export_path = os.path.join(self.export_csv_path,\
                "export", f"chute_{str(chute)}", f"cam_{str(cam)}", "fall")
        else:
            retrieval_path = os.path.join(self.data_path, "no_fall")
            export_path = os.path.join(self.export_csv_path,\
                "export", f"chute_{str(chute)}", f"cam_{str(cam)}", "no_fall")

        #Name of the video
        frame_name = f"chute{chute:02d}_cam{cam}_frames_{start}_{end}_"

        #Set the idx frame start and end
        idx_frame_start = start + self.frame_idx_start - 1
        idx_frame_end = start + self.frame_idx_start + self.frames_export - 1

        #Iterate across each index
        for i in np.arange(idx_frame_start, idx_frame_end):

            # Read the orig image and flow image
            orig_image_path = os.path.join(retrieval_path, frame_name + f"_{i}.jpg")
            image = np.array(Image.open(orig_image_path))

            # Read the .npy file
            optical_flow_path = os.path.join(retrieval_path, frame_name + f"_{i}.npy")
            flow = np.load(optical_flow_path)

            # Create a download path for orig image and flow image
            down_img_path = os.path.join(export_path, frame_name + f"_{i}_orig_frame.jpg")
            down_flow_path = os.path.join(export_path, frame_name + f"_{i}_flow_frame.jpg")

            # If it doesn't exist, download it.
            if not os.path.exists(down_img_path):
                Image.fromarray(image).save(down_img_path, quality=95)
            if not os.path.exists(down_flow_path):
                Image.fromarray(flow).save(down_flow_path, quality = 95)

### 3. Obtain the frames.

In [55]:
# Create a class
exporter = ExportFrames(dataframe=df_metadata,\
    dataloader=test_dataset, export_csv_path = os.getcwd(),
    data_path=OF_DATA_PATH,frame_idx_start=15)

In [56]:
exporter.export_frames()